In [31]:
# Verify required packages and imports
import torch
import torch.nn as nn
import sklearn
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.calibration import CalibrationDisplay
from sklearn.metrics import brier_score_loss
from pathlib import Path
import os

print(f"✓ PyTorch {torch.__version__}")
print(f"✓ scikit-learn {sklearn.__version__}")
print(f"✓ NumPy {np.__version__}")
print(f"✓ Pandas {pd.__version__}")
print(f"✓ Matplotlib {matplotlib.__version__}")

✓ PyTorch 2.10.0+cpu
✓ scikit-learn 1.8.0
✓ NumPy 2.4.2
✓ Pandas 3.0.0
✓ Matplotlib 3.10.8


# Calibration Curves

## Overview
Calibration curves (reliability diagrams) assess how well model confidence aligns with true accuracy. A perfectly calibrated model's curve follows the diagonal (y=x).

### Why Calibration Matters for AFib Detection

In clinical practice, AFib detection models produce probability scores that physicians use to make decisions:
- A score of **0.8 should mean ~80% likelihood of AFib** – not just a classification threshold
- **Overconfident models** (predictions > true frequency):
  - Risk: Unnecessary referrals, excessive monitoring, patient anxiety
  - Clinical consequence: Cost and burden increase without improved outcomes
- **Underconfident models** (predictions < true frequency):
  - Risk: Missed AFib cases, delayed treatment
  - Clinical consequence: Potential serious cardiac events if AF goes undetected

### What the Calibration Curve Shows

- **Curve above diagonal**: Model is **under-confident** (predicted probabilities < actual outcome frequency)
- **Curve below diagonal**: Model is **over-confident** (predicted probabilities > actual outcome frequency)
- **Curve on diagonal**: **Perfect calibration** (ideal scenario)
- **Brier Score**: Mean squared error between predicted probability and true label (0=perfect, 1=worst)

### Resolution Analysis

We examine 4 sampling rates (62.5Hz, 125Hz, 250Hz, 500Hz) because:
- **Lower frequencies (62.5Hz)**: Minimal storage, fast inference, clinically sufficient for AFib detection
- **Higher frequencies (500Hz)**: More signal detail, potential improved accuracy but increased complexity


## Setup: Imports and Configuration

In [32]:
import os
from pathlib import Path
import torch

# Project structure: AF_detection_vs_freq (3 levels up)
PROJECT_ROOT = Path.cwd().parent.parent.parent
CHECKPOINT_PATH = PROJECT_ROOT / "experiment_testing" / "mz-testing" / "checkpoints" / "ptbxl"
DATA_PATH = PROJECT_ROOT / "data"  # ✅ FIXED: Points to /data folder with resolution subfolders

print(f"📁 Notebook is here: {Path.cwd()}")
print(f"🎯 Project root: {PROJECT_ROOT}")
print(f"✅ Checkpoint path: {CHECKPOINT_PATH} -> {CHECKPOINT_PATH.exists()}")
print(f"✅ Data path: {DATA_PATH} -> {DATA_PATH.exists()}")

# Verify folders
res_folders = sorted([p.name for p in CHECKPOINT_PATH.glob("*hz")]) if CHECKPOINT_PATH.exists() else []
model_folders = sorted([p.name for p in (CHECKPOINT_PATH / "62hz").glob("*")]) if (CHECKPOINT_PATH / "62hz").exists() else []
data_folders = sorted([p.name for p in DATA_PATH.glob("*hz")]) if DATA_PATH.exists() else []
print(f"✓ Resolution folders in checkpoints: {res_folders}")
print(f"✓ Model folders: {model_folders}")
print(f"✓ Data resolution folders: {data_folders}")

# Resolution and Model mappings
RESOLUTIONS = {
    "62hz": "62.5Hz",
    "125hz": "125Hz",
    "250hz": "250Hz",
    "500hz": "500Hz"
}

MODELS = {
    "cnn1d": "1D-CNN",              # ✅ FIXED: cnn1d not cnn
    "cnn_lstm": "Hybrid CNN+LSTM"
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Device: {device}")

📁 Notebook is here: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\experiment_testing\adrian-testing\calibration_curves
🎯 Project root: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq
✅ Checkpoint path: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\experiment_testing\mz-testing\checkpoints\ptbxl -> True
✅ Data path: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\data -> True
✓ Resolution folders in checkpoints: ['125hz', '250hz', '500hz', '62hz']
✓ Model folders: ['cnn1d', 'cnn_lstm']
✓ Data resolution folders: ['100hz', '125hz', '250hz', '500hz', '62hz']
🖥️ Device: cpu


## 📁 File Structure & Paths

**Checkpoints** (will be checked):
- `experiment_testing/mz-testing/checkpoints/ptbxl/{62hz,125hz,250hz,500hz}/{cnn1d,cnn_lstm}/fold_1/best.pt`

**Test Labels** (will be checked):
- `prepared_data/{62hz,125hz,250hz,500hz}/test.pt` (preferred)
- `prepared_data/{62hz,125hz,250hz,500hz}/sample_*.csv` (fallback)

**Output** (will be generated):
- 8 PNG files: `calibration_{model}_{resolution}.png`


In [33]:
import pandas as pd
import numpy as np

def load_test_labels(res_folder):
    """Load test labels from prepared_data/{res_folder}/best.pt"""
    res_display = RESOLUTIONS.get(res_folder, res_folder)
    data_dir = DATA_PATH / res_folder
    
    print(f"    📂 Checking: {data_dir}")
    
    # Try best.pt
    test_pt_path = data_dir / "best.pt"
    print(f"    🔍 Trying: {test_pt_path.name}")
    if test_pt_path.exists():
        try:
            test_data = torch.load(test_pt_path, map_location=device)
            if isinstance(test_data, dict) and 'labels' in test_data:
                y_true = test_data['labels']
                if isinstance(y_true, torch.Tensor):
                    y_true = y_true.cpu().numpy()
                print(f"    ✓ Loaded labels from best.pt ({len(y_true)} samples)")
                return y_true
        except Exception as e:
            print(f"    ✗ Error loading best.pt: {e}")
    else:
        print(f"    ✗ best.pt not found at {test_pt_path}")
    

def load_checkpoint_and_predict(model_folder, res_folder):
    """Load checkpoint and extract predictions."""
    checkpoint_path = CHECKPOINT_PATH / res_folder / model_folder / "fold_1" / "best.pt"
    print(f"    🔍 Checkpoint: {checkpoint_path}")
    
    if not checkpoint_path.exists():
        print(f"    ✗ NOT FOUND: {checkpoint_path}")
        return None
    
    try:
        checkpoint = torch.load(checkpoint_path, map_location=device)
        checkpoint_keys = list(checkpoint.keys()) if isinstance(checkpoint, dict) else type(checkpoint).__name__
        print(f"    ✓ Loaded checkpoint (type: {type(checkpoint).__name__})")
        
        if isinstance(checkpoint, dict) and 'predictions' in checkpoint:
            y_probs = checkpoint['predictions']
            if isinstance(y_probs, torch.Tensor):
                y_probs = y_probs.cpu().numpy()
            print(f"    ✓ Extracted predictions ({len(y_probs)} samples)")
            return y_probs
        
        elif isinstance(checkpoint, dict) and ('model_state_dict' in checkpoint or 'state_dict' in checkpoint):
            print(f"    ℹ State dict detected (model instantiation needed)")
            return None
        
        elif isinstance(checkpoint, (torch.Tensor, np.ndarray)):
            y_probs = checkpoint
            if isinstance(y_probs, torch.Tensor):
                y_probs = y_probs.cpu().numpy()
            print(f"    ✓ Extracted predictions ({len(y_probs)} samples)")
            return y_probs
        
        else:
            print(f"    ⚠ Unknown checkpoint structure: {type(checkpoint)}")
            return None
    
    except Exception as e:
        print(f"    ✗ Error: {e}")
        return None


def get_calibration_data(res_folder, model_folder):
    """Get y_true and y_probs with detailed debug output."""
    print(f"\n  📊 {MODELS[model_folder]} @ {RESOLUTIONS[res_folder]}")
    print(f"  {'='*70}")
    
    # Load test labels
    print(f"  [1/2] Loading test labels...")
    y_true = load_test_labels(res_folder)
    if y_true is None:
        return None, None
    
    # Load predictions from checkpoint
    print(f"  [2/2] Loading checkpoint predictions...")
    y_probs = load_checkpoint_and_predict(model_folder, res_folder)
    if y_probs is None:
        return None, None
    
    # Validate shapes match
    if len(y_true) != len(y_probs):
        print(f"  ✗ Shape mismatch: y_true ({len(y_true)}) vs y_probs ({len(y_probs)})")
        return None, None
    
    print(f"  ✓ Loaded {len(y_true)} samples")
    return y_true, y_probs


## Model Architecture Classes

In [34]:

# Define CNN1D Architecture (from mz-testing/models/cnn1d.py)
class CNN1D(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(in_channels, 32, kernel_size=7, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            nn.Conv1d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


# Define CNN+LSTM Architecture (from mz-testing/models/cnn_lstm.py)
class ShortcutConvBlock1D(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=10, dropout=0.3, pool_kernel=2):
        super().__init__()
        if out_ch % 2 != 0:
            raise ValueError("out_ch must be even")
        self.bn = nn.BatchNorm1d(in_ch)
        self.conv = nn.Conv1d(in_channels=in_ch, out_channels=out_ch, 
                              kernel_size=kernel_size, padding=kernel_size // 2, bias=False)
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(dropout)
        self.avgpool = nn.AvgPool1d(kernel_size=pool_kernel, stride=pool_kernel)
        self.maxpool = nn.MaxPool1d(kernel_size=pool_kernel, stride=pool_kernel)

    def forward(self, x):
        x = self.bn(x)
        x = self.conv(x)
        x = self.act(x)
        x = self.drop(x)
        c_half = x.size(1) // 2
        x_main = x[:, :c_half, :]
        x_short = x[:, c_half:, :]
        x_main = self.avgpool(x_main)
        x_short = self.maxpool(x_short)
        return torch.cat([x_main, x_short], dim=1)


class CNN_LSTM_ECG(nn.Module):
    def __init__(self, in_channels, num_classes, dropout=0.3):
        super().__init__()
        ch = [32, 32, 64, 64, 128, 128, 256, 256]
        blocks = []
        c_in = in_channels
        for c_out in ch:
            blocks.append(ShortcutConvBlock1D(in_ch=c_in, out_ch=c_out, 
                                             kernel_size=10, dropout=dropout, pool_kernel=2))
            c_in = c_out
        self.cnn = nn.Sequential(*blocks)
        self.lstm = nn.LSTM(input_size=ch[-1], hidden_size=128, 
                           num_layers=1, batch_first=True, dropout=0.0, bidirectional=False)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = x.permute(0, 2, 1)
        out, _ = self.lstm(x)
        out = out.mean(dim=1)
        return self.classifier(out)

print("✅ Model architectures loaded")


✅ Model architectures loaded


In [35]:
def generate_calibration_curve(y_true, y_probs, model_name, resolution, output_dir="."):
    """
    Generate and save a calibration curve PNG.
    
    Args:
        y_true: np.array of true binary labels [n_samples]
        y_probs: np.array of predicted AFib probabilities [n_samples], range [0, 1]
        model_name: str, display name e.g., "1D-CNN"
        resolution: str, display name e.g., "62.5Hz"
        output_dir: directory to save PNG
    
    Returns:
        dict with metrics: {model, resolution, brier, n_samples, mean_pred, mean_true, filepath}
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Ensure probabilities are in valid range
    y_probs = np.clip(y_probs, 0, 1)
    
    # Compute Brier score
    brier = brier_score_loss(y_true, y_probs)
    
    # Compute statistics
    n_samples = len(y_true)
    mean_pred = np.mean(y_probs)
    mean_true = np.mean(y_true)
    
    # Warn if small sample size
    if n_samples < 50:
        print(f"  ⚠ Warning: Small sample size ({n_samples} < 50)")
    
    print(f"  Brier Score: {brier:.6f}")
    print(f"  Mean Pred: {mean_pred:.4f} | Mean True: {mean_true:.4f} | N={n_samples}")
    print(f"  Class dist: {np.sum(y_true==0)} normal, {np.sum(y_true==1)} AFib")
    
    # Create figure with calibration curve
    fig, ax = plt.subplots(figsize=(10, 5))
    
    # Generate calibration curve using sklearn
    display = CalibrationDisplay.from_predictions(
        y_true,
        y_probs,
        n_bins=10,
        strategy="uniform",
        ax=ax,
        color="steelblue",
        name="Model"
    )
    
    # Note: CalibrationDisplay already adds the perfect calibration diagonal
    
    # Customize plot
    ax.set_title(f"Calibration Curve – {model_name}, {resolution}", 
                 fontsize=14, fontweight="bold")
    ax.set_xlabel("Mean Predicted Probability (AFib)", fontsize=12)
    ax.set_ylabel("True Positive Fraction (AFib)", fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(loc="lower right", fontsize=10)
    
    # Add Brier score annotation
    textstr = f"Brier Score: {brier:.6f}"
    ax.text(0.98, 0.02, textstr, transform=ax.transAxes, 
            fontsize=10, verticalalignment='bottom', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    
    # Save figure (safe filename)
    model_safe = model_name.replace("+", "").replace(" ", "_")
    res_safe = resolution.replace(".", "").replace(" ", "")
    output_filename = f"calibration_{model_safe}_{res_safe}.png"
    output_path = output_dir / output_filename
    
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"  ✓ Saved: {output_filename}")
    plt.close()
    
    return {
        "Model": model_name,
        "Resolution": resolution,
        "Brier Score": brier,
        "N Samples": n_samples,
        "Mean Pred Prob": mean_pred,
        "Mean True Label": mean_true,
        "Filepath": str(output_path)
    }


## Generate Individual Calibration Curves for All Model-Resolution Combinations
This cell generates calibration curves for each of the 12 combinations (3 models × 4 resolutions).
Each curve is saved as a standalone PNG file with clear labels and perfect diagonal reference.

In [36]:
# Define test data paths directly based on known structure
def find_test_data_files():
    """Get test data files from data/{resolution}/test/test.pt"""
    test_files = {}
    project_root = Path.cwd().parent.parent.parent
    data_root = project_root / "data"
    
    print("🔍 Loading test data files from /data folder...")
    
    for res in RESOLUTIONS.keys():
        test_path = data_root / res / "test" / "test.pt"
        if test_path.exists():
            test_files[res] = test_path
            print(f"  ✅ Found {res}: {test_path}")
        else:
            print(f"  ❌ Missing {res}: {test_path}")
    
    return test_files


# Find test data
test_data_files = find_test_data_files()
print(f"\n📦 Total test data files found: {len(test_data_files)}")
for res, path in test_data_files.items():
    print(f"  {RESOLUTIONS[res]}: {path.exists()}")

🔍 Loading test data files from /data folder...
  ✅ Found 62hz: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\data\62hz\test\test.pt
  ✅ Found 125hz: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\data\125hz\test\test.pt
  ✅ Found 250hz: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\data\250hz\test\test.pt
  ✅ Found 500hz: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\data\500hz\test\test.pt

📦 Total test data files found: 4
  62.5Hz: True
  125Hz: True
  250Hz: True
  500Hz: True


## STEP 3: Main Loop Execution

## STEP 1: Inspect Test Data Structure

Before loading all test data, let's inspect one file to understand its structure (keys, shapes, types).

In [37]:
from pathlib import Path
import torch

# Inspect one test file to see its keys and shapes
print("🔍 Inspecting test data structure...")
print("=" * 80)

# Try to load a sample test file (125hz as example)
sample_test_path = PROJECT_ROOT / "data" / "125hz" / "test" / "test.pt"
if not sample_test_path.exists():
    # Try 62hz instead
    sample_test_path = PROJECT_ROOT / "data" / "62hz" / "test" / "test.pt"

print(f"Loading: {sample_test_path}")
print(f"Exists: {sample_test_path.exists()}")

if sample_test_path.exists():
    test_data = torch.load(sample_test_path, map_location="cpu")
    print(f"\nType: {type(test_data)}")
    
    if isinstance(test_data, dict):
        print(f"Keys: {list(test_data.keys())}")
        print("\nDetails:")
        for k, v in test_data.items():
            try:
                v_type = type(v).__name__
                v_shape = getattr(v, 'shape', None)
                v_len = len(v) if hasattr(v, '__len__') else 'n/a'
                v_dtype = getattr(v, 'dtype', None)
                print(f"  '{k}':")
                print(f"    - type: {v_type}")
                print(f"    - shape: {v_shape}")
                print(f"    - length: {v_len}")
                print(f"    - dtype: {v_dtype}")
            except Exception as e:
                print(f"  '{k}': type={type(v)}, error={e}")
    else:
        print("⚠ Non-dict structure")
        print(f"Repr: {repr(test_data)[:300]}")
else:
    print("❌ Test file not found!")

print("=" * 80)

🔍 Inspecting test data structure...
Loading: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\data\125hz\test\test.pt
Exists: True

Type: <class 'dict'>
Keys: ['X', 'y', 'record_ids', 'patient_ids']

Details:
  'X':
    - type: Tensor
    - shape: torch.Size([2185, 12, 1250])
    - length: 2185
    - dtype: torch.float32
  'y':
    - type: Tensor
    - shape: torch.Size([2185])
    - length: 2185
    - dtype: torch.int64
  'record_ids':
    - type: list
    - shape: None
    - length: 2185
    - dtype: None
  'patient_ids':
    - type: list
    - shape: None
    - length: 2185
    - dtype: None


## Helper Function: Extract Features and Labels

This function explicitly checks for keys instead of using `or` operator, which causes RuntimeError with Tensors.

In [38]:
def extract_xy(test_data):
    """
    Extract features (X) and labels (y) from test data dictionary.
    
    ⚠ CRITICAL: Use explicit key checks, NOT 'or' operator with Tensors!
    The pattern `tensor_a or tensor_b` raises: "Boolean value of Tensor with more than one value is ambiguous"
    
    Args:
        test_data: dict with test data
        
    Returns:
        tuple (X, y_true) as Tensors
    """
    if not isinstance(test_data, dict):
        raise TypeError(f"Expected dict, got {type(test_data)}")
    
    # Extract labels with explicit key checks
    if "y" in test_data:
        y_true = test_data["y"]
    elif "labels" in test_data:
        y_true = test_data["labels"]
    else:
        raise KeyError(f"No label key found. Available keys: {list(test_data.keys())}")
    
    # Extract features with explicit key checks
    if "X" in test_data:
        X = test_data["X"]
    elif "x" in test_data:
        X = test_data["x"]
    elif "signals" in test_data:
        X = test_data["signals"]
    elif "data" in test_data:
        X = test_data["data"]
    else:
        raise KeyError(f"No feature key found. Available keys: {list(test_data.keys())}")
    
    return X, y_true

print("✅ Helper function loaded: extract_xy()")

✅ Helper function loaded: extract_xy()


## STEP 2: Load Models & Generate Calibration Curves

### ⚠ Bug Fix: RuntimeError with Tensor boolean evaluation

**Problem:**
```python
# ❌ BUGGY CODE:
y_true = test_data.get('y') or test_data.get('labels')
```

**Error:**
```
RuntimeError: Boolean value of Tensor with more than one value is ambiguous
```

**Root Cause:**
- Python's `or` operator evaluates the left operand first: `A or B`
- If `A` is a non-empty Tensor, Python tries to convert it to boolean
- Multi-element Tensors cannot be used in boolean context → RuntimeError

**Solution:**
```python
# ✅ FIXED CODE:
if "y" in test_data:
    y_true = test_data["y"]
elif "labels" in test_data:
    y_true = test_data["labels"]
else:
    raise KeyError(...)
```

We use the `extract_xy()` helper function that implements explicit key checks.

In [39]:
import os
import torch.nn as nn

# Initialize results storage
results_list = []
saved_files = []

# Storage for 2x2 comparison plots (collected by resolution and model)
y_true_by_res = {}  # {res_folder: y_true_array}
y_probs_by_res_model = {}  # {res_folder: {model_folder: y_probs_array}}

print("=" * 100)
print("🚀 MAIN LOOP: Load Models + Run Inference + Generate Calibration Curves")
print("=" * 100)

# Loop through all model-resolution combinations
for res_folder, res_display in RESOLUTIONS.items():
    for model_folder, model_display in MODELS.items():
        
        print(f"\n{'─'*100}")
        print(f"📊 {model_display} @ {res_display}")
        print(f"{'─'*100}")
        
        # Get checkpoint path
        checkpoint_path = CHECKPOINT_PATH / res_folder / model_folder / "fold_1" / "best.pt"
        print(f"[1/3] Checkpoint: {checkpoint_path}")
        print(f"      Exists: {checkpoint_path.exists()}")
        
        if not checkpoint_path.exists():
            print(f"      ❌ SKIPPED: Checkpoint not found")
            continue
        
        # Get test data path
        test_data_path = test_data_files.get(res_folder)
        if test_data_path is None:
            print(f"[2/3] Test Data: ❌ NOT FOUND for {res_folder}")
            continue
        print(f"[2/3] Test Data: {test_data_path}")
        print(f"      Exists: {test_data_path.exists()}")
        
        try:
            # Load test data
            print(f"[2b] Loading test data...")
            test_data = torch.load(test_data_path, map_location='cpu')
            
            # ✅ FIXED: Use explicit key checks instead of 'or' operator
            # OLD (BUGGY): y_true = test_data.get('y') or test_data.get('labels')
            # NEW: Use helper function with explicit key checks
            X_test, y_true = extract_xy(test_data)
            
            # Convert to proper format
            if isinstance(y_true, torch.Tensor):
                y_true = y_true.detach().cpu().numpy().ravel()
            else:
                y_true = np.array(y_true).ravel()
            
            if isinstance(X_test, torch.Tensor):
                X_test = X_test.detach().cpu()
            else:
                X_test = torch.tensor(X_test, dtype=torch.float32)
            
            print(f"      ✓ Loaded {len(y_true)} test samples")
            print(f"      X shape: {X_test.shape}, y shape: {y_true.shape}")
            print(f"      y range: [{y_true.min():.2f}, {y_true.max():.2f}]")
            
            # Load checkpoint and create model
            print(f"[3/3] Loading model and running inference...")
            checkpoint = torch.load(checkpoint_path, map_location='cpu')
            
            # Initialize appropriate model
            if model_folder == "cnn1d":
                model = CNN1D(in_channels=12, num_classes=2)
            elif model_folder == "cnn_lstm":
                model = CNN_LSTM_ECG(in_channels=12, num_classes=2)
            else:
                print(f"      ❌ Unknown model: {model_folder}")
                continue
            
            # Load state dict
            model.load_state_dict(checkpoint)
            model.eval()
            model = model.to('cpu')
            
            print(f"      ✓ Model loaded")
            
            # Run inference to get predictions
            with torch.no_grad():
                batch_size = 32
                y_probs_list = []
                
                for i in range(0, len(X_test), batch_size):
                    batch = X_test[i:i+batch_size]
                    if not isinstance(batch, torch.Tensor):
                        batch = torch.tensor(batch, dtype=torch.float32)
                    batch = batch.to('cpu')
                    
                    logits = model(batch)  # (B, num_classes)
                    
                    # Get probability for AFib class (class 1)
                    if logits.shape[-1] == 2:
                        probs = torch.softmax(logits, dim=-1)[:, 1]  # AFib is class 1
                    else:
                        probs = torch.sigmoid(logits.squeeze())
                    
                    y_probs_list.append(probs.cpu().numpy())
                
                y_probs = np.concatenate(y_probs_list, axis=0)
            
            print(f"      ✓ Inference complete: {len(y_probs)} predictions")
            print(f"      Prob range: [{np.min(y_probs):.4f}, {np.max(y_probs):.4f}]")
            
            # Ensure labels are binary (0 or 1)
            y_true_binary = (y_true > 0.5).astype(int) if y_true.dtype == np.float64 or y_true.dtype == np.float32 else y_true.astype(int)
            
            print(f"      Label distribution: {np.sum(y_true_binary==0)} normal, {np.sum(y_true_binary==1)} AFib")
            
            # Store data for comparison plots
            if res_folder not in y_true_by_res:
                y_true_by_res[res_folder] = y_true_binary
            if res_folder not in y_probs_by_res_model:
                y_probs_by_res_model[res_folder] = {}
            y_probs_by_res_model[res_folder][model_folder] = y_probs
            
            # Generate calibration curve
            print(f"      🎨 Generating calibration curve...")
            result = generate_calibration_curve(y_true_binary, y_probs, model_display, res_display, output_dir=".")
            results_list.append(result)
            
            # Track saved file
            model_safe = model_display.replace("+", "").replace(" ", "_")
            res_safe = res_display.replace(".", "").replace(" ", "")
            output_filename = f"calibration_{model_safe}_{res_safe}.png"
            saved_files.append(output_filename)
            
            print(f"      ✅ SUCCESS: {output_filename}")
            
        except Exception as e:
            print(f"      ❌ ERROR: {e}")
            import traceback
            traceback.print_exc()
            continue

print(f"\n{'='*100}")
print(f"✅ COMPLETED: Generated {len(saved_files)} calibration curves")
print(f"{'='*100}")

🚀 MAIN LOOP: Load Models + Run Inference + Generate Calibration Curves

────────────────────────────────────────────────────────────────────────────────────────────────────
📊 1D-CNN @ 62.5Hz
────────────────────────────────────────────────────────────────────────────────────────────────────
[1/3] Checkpoint: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\experiment_testing\mz-testing\checkpoints\ptbxl\62hz\cnn1d\fold_1\best.pt
      Exists: True
[2/3] Test Data: c:\Users\adria\VSCode_Projects\bachelorWork\AF_detection_vs_freq\data\62hz\test\test.pt
      Exists: True
[2b] Loading test data...
      ✓ Loaded 2185 test samples
      X shape: torch.Size([2185, 12, 620]), y shape: (2185,)
      y range: [0.00, 1.00]
[3/3] Loading model and running inference...
      ✓ Model loaded
      ✓ Inference complete: 2185 predictions
      Prob range: [0.0000, 1.0000]
      Label distribution: 1894 normal, 291 AFib
      🎨 Generating calibration curve...
  Brier Score: 0.010784
  

## Generate 2×2 Comparison Plots

Create one plot per model showing all 4 resolutions side-by-side for easy comparison.

In [40]:
print("\n" + "=" * 100)
print("🎨 GENERATING 2×2 COMPARISON PLOTS")
print("=" * 100)

# Generate one 2×2 plot per model (all resolutions)
comparison_files = []

for model_folder, model_display in MODELS.items():
    print(f"\n📊 Creating comparison plot for {model_display}...")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    axes = axes.flatten()
    
    for idx, (res_folder, res_display) in enumerate(RESOLUTIONS.items()):
        ax = axes[idx]
        
        # Check if we have data for this resolution and model
        has_data = (res_folder in y_true_by_res and 
                    res_folder in y_probs_by_res_model and 
                    model_folder in y_probs_by_res_model.get(res_folder, {}))
        
        if not has_data:
            ax.text(0.5, 0.5, f"No data for {res_display}", 
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_xlabel("")
            ax.set_ylabel("")
            continue
        
        # Get data
        y_true = y_true_by_res[res_folder]
        y_probs = y_probs_by_res_model[res_folder][model_folder]
        
        # Compute metrics
        brier = brier_score_loss(y_true, y_probs)
        
        # Generate calibration curve on this subplot
        display = CalibrationDisplay.from_predictions(
            y_true,
            y_probs,
            n_bins=10,
            strategy="uniform",
            ax=ax,
            color="steelblue",
            name=None  # Don't add legend label
        )
        
        # Customize subplot
        ax.set_title(f"{res_display} (n={len(y_true)})", fontsize=13, fontweight="bold")
        ax.set_xlabel("Mean Predicted Probability (AFib)", fontsize=11)
        ax.set_ylabel("True Positive Fraction (AFib)", fontsize=11)
        ax.grid(True, alpha=0.3)
        ax.legend().set_visible(False)  # Remove legend for cleaner look
        
        # Add Brier score annotation
        ax.text(0.98, 0.02, f"Brier: {brier:.4f}", transform=ax.transAxes,
                fontsize=10, verticalalignment='bottom', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Set overall title
    fig.suptitle(f"Calibration Curves – {model_display} (All Resolutions)", 
                 fontsize=16, fontweight="bold", y=0.995)
    plt.tight_layout()
    
    # Save figure
    model_safe = model_display.replace("+", "").replace(" ", "_")
    output_filename = f"calibration_comparison_{model_safe}.png"
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    comparison_files.append(output_filename)
    print(f"  ✅ Saved: {output_filename}")
    plt.close()

print(f"\n{'='*100}")
print(f"✅ COMPLETED: Generated {len(comparison_files)} comparison plots")
print(f"{'='*100}")


🎨 GENERATING 2×2 COMPARISON PLOTS

📊 Creating comparison plot for 1D-CNN...
  ✅ Saved: calibration_comparison_1D-CNN.png

📊 Creating comparison plot for Hybrid CNN+LSTM...
  ✅ Saved: calibration_comparison_Hybrid_CNNLSTM.png

✅ COMPLETED: Generated 2 comparison plots


## Generate Overlay Comparison Plots

Create one plot per model with all 4 resolutions overlaid as distinct colored lines for direct comparison.

In [45]:
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable

print("\n" + "=" * 100)
print("🎨 GENERATING PUBLICATION-READY OVERLAY COMPARISON PLOTS")
print("=" * 100)

# Color gradient: bright yellow (62.5Hz) to dark red (500Hz)
colors = {
    "62hz": "#FFFF00",      # Bright yellow
    "125hz": "#FFA500",     # Orange
    "250hz": "#FF6347",     # Tomato/Red-orange
    "500hz": "#8B0000"      # Dark red
}

overlay_files = []
individual_axes = []  # Store axes for combined figure

# Model name mapping for publication
pub_model_names = {
    "cnn1d": "1D-CNN",
    "cnn_lstm": "CNN-LSTM Hybrid"
}

# STEP 1: Generate individual plots (IEEE quality)
for model_folder, model_display in MODELS.items():
    print(f"\n📊 Creating publication-ready overlay plot for {model_display}...")
    
    # IEEE paper column width: 12x5 inches
    fig, ax = plt.subplots(figsize=(12, 5))
    
    for res_folder, res_display in RESOLUTIONS.items():
        # Check if we have data
        has_data = (res_folder in y_true_by_res and 
                    res_folder in y_probs_by_res_model and 
                    model_folder in y_probs_by_res_model.get(res_folder, {}))
        
        if not has_data:
            print(f"  ⚠ No data for {res_display}")
            continue
        
        # Get data
        y_true = y_true_by_res[res_folder]
        y_probs = y_probs_by_res_model[res_folder][model_folder]
        
        # Compute Brier score
        brier = brier_score_loss(y_true, y_probs)
        
        # Plot calibration curve with custom color
        color = colors.get(res_folder, "blue")
        display = CalibrationDisplay.from_predictions(
            y_true,
            y_probs,
            n_bins=10,
            strategy="uniform",
            ax=ax,
            color=color,
            name=f"{res_display} (Brier: {brier:.4f})",
            linewidth=2.5,
            marker='o',
            markersize=6
        )
        
        print(f"  ✓ Plotted {res_display} with Brier: {brier:.4f}")
    
    # ✅ IEEE PUBLICATION FORMATTING
    # Title with clinical context
    pub_name = pub_model_names.get(model_folder, model_display)
    ax.set_title(f"Calibration Curves – {pub_name} (AFib Positive Class)", 
                 fontsize=14, fontweight="bold", pad=15)
    
    # Precise axis labels
    ax.set_xlabel("Mean Predicted Probability (AFib)", fontsize=12)
    ax.set_ylabel("Fraction of Positives (AFib)", fontsize=12)
    
    # Legend in upper left (less overlap)
    ax.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98), 
             fontsize=10, framealpha=0.95, edgecolor='gray')
    
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    
    # Perfect calibration diagonal
    ax.plot([0, 1], [0, 1], "k--", linewidth=2, alpha=0.5, zorder=1)
    
    plt.tight_layout()
    
    # Save with IEEE quality settings
    model_safe = model_display.replace("+", "").replace(" ", "_").replace("-", "").lower()
    output_filename = f"calibration_{model_safe}.png"
    plt.savefig(output_filename, dpi=300, bbox_inches='tight', facecolor='white')
    overlay_files.append(output_filename)
    print(f"  ✅ Saved: {output_filename}")
    
    # Store axis for combined figure
    individual_axes.append((model_folder, model_display, ax))
    plt.close()

print(f"\n{'='*100}")
print(f"✅ Individual plots completed: {len(overlay_files)} files")
print(f"{'='*100}")

# STEP 2: Generate combined figure* (both models side-by-side)
print("\n📊 Creating combined figure* for IEEE paper...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
axes_list = [ax1, ax2]

for idx, (model_folder, model_display) in enumerate(MODELS.items()):
    ax = axes_list[idx]
    
    for res_folder, res_display in RESOLUTIONS.items():
        has_data = (res_folder in y_true_by_res and 
                    res_folder in y_probs_by_res_model and 
                    model_folder in y_probs_by_res_model.get(res_folder, {}))
        
        if not has_data:
            continue
        
        y_true = y_true_by_res[res_folder]
        y_probs = y_probs_by_res_model[res_folder][model_folder]
        brier = brier_score_loss(y_true, y_probs)
        color = colors.get(res_folder, "blue")
        
        display = CalibrationDisplay.from_predictions(
            y_true,
            y_probs,
            n_bins=10,
            strategy="uniform",
            ax=ax,
            color=color,
            name=f"{res_display} (Brier: {brier:.4f})",
            linewidth=2.5,
            marker='o',
            markersize=6
        )
    
    # Format each subplot
    pub_name = pub_model_names.get(model_folder, model_display)
    ax.set_title(f"{pub_name}", fontsize=13, fontweight="bold", pad=10)
    ax.set_xlabel("Mean Predicted Probability (AFib)", fontsize=11)
    ax.set_ylabel("Fraction of Positives (AFib)", fontsize=11)
    ax.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98), 
             fontsize=9, framealpha=0.95, edgecolor='gray')
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.plot([0, 1], [0, 1], "k--", linewidth=2, alpha=0.5, zorder=1)

plt.tight_layout()

# Save combined figure
combined_filename = "fig_calibration.png"
plt.savefig(combined_filename, dpi=300, bbox_inches='tight', facecolor='white')
print(f"  ✅ Saved combined figure: {combined_filename}")
plt.close()

print(f"\n{'='*100}")
print("✅ PUBLICATION-READY PLOTS GENERATED")
print(f"{'='*100}")

# LaTeX-ready summary
print("\n📄 LaTeX-ready filenames for \\includegraphics:")
print("-" * 80)
print(f"fig_calibration.png          % figure* (both models side-by-side)")
for fname in overlay_files:
    model_type = fname.split('_')[1].replace('.png', '')
    print(f"{fname:<28} % Individual {model_type.upper()}")
print("=" * 80)

print("\n📊 Files generated:")
print(f"  • Combined figure: {combined_filename}")
print(f"  • Individual plots: {', '.join(overlay_files)}")


🎨 GENERATING PUBLICATION-READY OVERLAY COMPARISON PLOTS

📊 Creating publication-ready overlay plot for 1D-CNN...
  ✓ Plotted 62.5Hz with Brier: 0.0108
  ✓ Plotted 125Hz with Brier: 0.0130
  ✓ Plotted 250Hz with Brier: 0.0139
  ✓ Plotted 500Hz with Brier: 0.0289
  ✅ Saved: calibration_1dcnn.png

📊 Creating publication-ready overlay plot for Hybrid CNN+LSTM...
  ✓ Plotted 62.5Hz with Brier: 0.0155
  ✓ Plotted 125Hz with Brier: 0.0126
  ✓ Plotted 250Hz with Brier: 0.0160
  ✓ Plotted 500Hz with Brier: 0.0152
  ✅ Saved: calibration_hybrid_cnnlstm.png

✅ Individual plots completed: 2 files

📊 Creating combined figure* for IEEE paper...
  ✅ Saved combined figure: fig_calibration.png

✅ PUBLICATION-READY PLOTS GENERATED

📄 LaTeX-ready filenames for \includegraphics:
--------------------------------------------------------------------------------
fig_calibration.png          % figure* (both models side-by-side)
calibration_1dcnn.png        % Individual 1DCNN
calibration_hybrid_cnnlstm.png % In

## Summary: Saved PNG Files

## 📊 Expected Output

**Individual Calibration Curve PNG files** (8 files - 1 per model-resolution combo):

1. calibration_1DCNN_625Hz.png
2. calibration_1DCNN_125Hz.png
3. calibration_1DCNN_250Hz.png
4. calibration_1DCNN_500Hz.png
5. calibration_HybridCNNLSTM_625Hz.png
6. calibration_HybridCNNLSTM_125Hz.png
7. calibration_HybridCNNLSTM_250Hz.png
8. calibration_HybridCNNLSTM_500Hz.png

**2×2 Comparison Grid PNG files** (2 files - 1 per model with 2×2 grid):

9. calibration_comparison_1D-CNN.png (all 4 resolutions in 2×2 grid)
10. calibration_comparison_HybridCNNLSTM.png (all 4 resolutions in 2×2 grid)

**Overlay Comparison PNG files** (2 files - 1 per model with all resolutions overlaid):

11. calibration_overlay_1D-CNN.png (all 4 resolutions with distinct colors)
12. calibration_overlay_HybridCNNLSTM.png (all 4 resolutions with distinct colors)

### File Descriptions

**Individual PNG** (8 files):
- **Content**: Single calibration curve per model-resolution combination
- **Colors**: Blue curve with diagonal reference
- **Size**: 10×5 inches, 300 dpi
- **Use**: Detailed view of each model-resolution pair

**2×2 Grid PNG** (2 files):
- **Content**: All 4 resolutions for one model in a 2×2 grid layout
- **Consistent scales**: Easy visual comparison
- **Colors**: Blue curves with consistent scaling
- **Size**: 14×12 inches, 300 dpi
- **Use**: Comparing how a single model performs across different resolutions

**Overlay PNG** (2 files):
- **Content**: All 4 resolutions for one model overlaid on the same plot
- **Color gradient**: 
  - 62.5Hz: Bright Yellow (#FFFF00)
  - 125Hz: Orange (#FFA500)
  - 250Hz: Tomato Red (#FF6347)
  - 500Hz: Dark Red (#8B0000)
- **Size**: 12×8 inches, 300 dpi
- **Use**: Direct visual comparison of resolution impact on calibration, following each resolution's colored line

In [41]:
print("\nGenerated individual PNG files:")
print("-" * 80)
for i, fname in enumerate(saved_files, 1):
    print(f"{i:2d}. {fname}")

print("\n" + "=" * 80)
print(f"Total individual files saved: {len(saved_files)}")
print("=" * 80)

if 'comparison_files' in dir() and len(comparison_files) > 0:
    print("\nGenerated comparison PNG files:")
    print("-" * 80)
    for i, fname in enumerate(comparison_files, 1):
        print(f"{i:2d}. {fname}")
    print("\n" + "=" * 80)
    print(f"Total comparison files saved: {len(comparison_files)}")
    print("=" * 80)


Generated individual PNG files:
--------------------------------------------------------------------------------
 1. calibration_1D-CNN_625Hz.png
 2. calibration_Hybrid_CNNLSTM_625Hz.png
 3. calibration_1D-CNN_125Hz.png
 4. calibration_Hybrid_CNNLSTM_125Hz.png
 5. calibration_1D-CNN_250Hz.png
 6. calibration_Hybrid_CNNLSTM_250Hz.png
 7. calibration_1D-CNN_500Hz.png
 8. calibration_Hybrid_CNNLSTM_500Hz.png

Total individual files saved: 8

Generated comparison PNG files:
--------------------------------------------------------------------------------
 1. calibration_comparison_1D-CNN.png
 2. calibration_comparison_Hybrid_CNNLSTM.png

Total comparison files saved: 2


## 📈 Metrics Guide

### Brier Score (Calibration Error)
- **Range**: 0 (perfect) to 1 (worst)
- **Typical clinical models**: 0.1–0.3
- **Interpretation**: Mean squared error between predicted probability and true label

### Calibration Curve Positioning
- **Curve ABOVE diagonal**: Model is **under-confident** (predicted prob < true frequency)
- **Curve BELOW diagonal**: Model is **over-confident** (predicted prob > true frequency)  
- **Curve ON diagonal**: **Perfect calibration** (ideal)

### AFib Detection Relevance
- Under-confident: Risk of missing AFib cases (safety concern)
- Over-confident: Unnecessary referrals (cost concern)
- Well-calibrated: Clinicians can trust probability scores


## Bonus: 2×2 Subplots – All Resolutions Per Model


In [42]:
def plot_model_subplots_2x2(model_name, y_true_by_res, predictions_by_res):
    """
    Plot 2×2 grid of calibration curves for all 4 resolutions of a single model.
    
    Args:
        model_name: str, "1D-CNN" or "Hybrid CNN+LSTM"
        y_true_by_res: dict {resolution: y_true_array}
        predictions_by_res: dict {resolution: {model_name: y_probs_array}}
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    axes = axes.flatten()
    
    for idx, resolution in enumerate(RESOLUTIONS):
        ax = axes[idx]
        
        if resolution not in y_true_by_res or model_name not in predictions_by_res.get(resolution, {}):
            ax.text(0.5, 0.5, f"No data for {resolution}Hz", ha='center', va='center', transform=ax.transAxes)
            ax.set_xticks([])
            ax.set_yticks([])
            continue
        
        y_true = y_true_by_res[resolution]
        y_probs = predictions_by_res[resolution][model_name]
        
        # Compute metrics
        brier = brier_score_loss(y_true, y_probs)
        mean_pred = np.mean(y_probs)
        mean_true = np.mean(y_true)
        
        # Generate calibration curve on this subplot
        display = CalibrationDisplay.from_predictions(
            y_true,
            y_probs,
            n_bins=10,
            strategy="uniform",
            ax=ax,
            color="steelblue"
        )
        
        # Add perfect calibration diagonal
        ax.plot([0, 1], [0, 1], "k--", linewidth=2, alpha=0.7)
        
        # Title and labels
        ax.set_title(f"{resolution}Hz (n={len(y_true)})", fontsize=12, fontweight="bold")
        ax.set_xlabel("Mean Predicted Probability", fontsize=10)
        ax.set_ylabel("True Positive Fraction", fontsize=10)
        ax.grid(True, alpha=0.3)
        
        # Add Brier score
        ax.text(0.98, 0.02, f"Brier: {brier:.4f}", transform=ax.transAxes,
                fontsize=9, verticalalignment='bottom', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    fig.suptitle(f"Calibration Curves – {model_name} (All Resolutions)", 
                 fontsize=16, fontweight="bold", y=0.995)
    plt.tight_layout()
    
    # Save figure
    output_filename = f"calibration_subplots_{model_name.replace('+', '_').replace(' ', '')}.png"
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {output_filename}")
    plt.show()


# Usage example (run after main loop):
# Collect data per resolution if needed
# y_true_by_res = {res: ... for res in RESOLUTIONS}
# predictions_by_res = {res: {model: ... for model in MODELS} for res in RESOLUTIONS}
# for model in MODELS:
#     plot_model_subplots_2x2(model, y_true_by_res, predictions_by_res)


## Summary Table: Calibration Metrics

In [43]:
# Create and display summary table
print("\n" + "=" * 120)
print("CALIBRATION METRICS SUMMARY TABLE")
print("=" * 120)

if len(results_list) == 0:
    print("⚠ No results generated. Check that prediction files exist.")
else:
    # Create DataFrame from results
    df_summary = pd.DataFrame(results_list)
    
    # Select and reorder columns for display
    display_cols = ["Model", "Resolution", "Brier Score", "N Samples", "Mean Pred Prob", "Mean True Label"]
    df_display = df_summary[display_cols].copy()
    
    # Sort by resolution then model
    res_order = ["62.5Hz", "125Hz", "250Hz", "500Hz"]
    df_display['Resolution'] = pd.Categorical(df_display['Resolution'], categories=res_order, ordered=True)
    df_display = df_display.sort_values(by=['Resolution', 'Model']).reset_index(drop=True)
    
    # Format for display
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print(df_display.to_string(index=False))
    
    print("\n" + "=" * 120)
    print("INTERPRETATION GUIDE")
    print("=" * 120)
    print("""
1. BRIER SCORE (lower is better):
   • Mean squared error between predicted probability and true label
   • Range: 0 (perfect) to 1 (worst)
   • Typical clinical models: 0.1–0.3
   • Interpretation: Measures calibration error and classification accuracy combined

2. MEAN PREDICTED PROB (AFib):
   • Average confidence the model assigns to AFib (class=1)
   • If much lower than "Mean True Label": Model is UNDERCONFIDENT
   • If much higher than "Mean True Label": Model is OVERCONFIDENT
   • Ideal: Close to "Mean True Label" value

3. MEAN TRUE LABEL (AFib):
   • True prevalence of AFib in test set
   • Indicates class balance (0.5 = balanced, <0.5 = AFib underrepresented)

4. CALIBRATION CURVE POSITIONING:
   • Above diagonal: Predicted probability < actual outcome frequency (underconfident)
   • Below diagonal: Predicted probability > actual outcome frequency (overconfident)
   • On diagonal: Perfect calibration (ideal)

CLINICAL RELEVANCE FOR AFib DETECTION:
   • A well-calibrated model enables clinicians to interpret confidence scores meaningfully
   • E.g., if model predicts 0.8 AFib probability, it should reflect ~80% actual AFib likelihood
   • Underconfidence → risk of missing AFib cases (safety concern)
   • Overconfidence → unnecessary referrals/treatments (cost/burden concern)
""")
    
    print("\n" + "=" * 120)
    print("AGGREGATE STATISTICS")
    print("=" * 120)
    brier_scores = df_display['Brier Score'].values
    print(f"Average Brier Score:        {np.mean(brier_scores):.6f}")
    print(f"  Min: {np.min(brier_scores):.6f} ({df_display.iloc[np.argmin(brier_scores)]['Model']} @ {df_display.iloc[np.argmin(brier_scores)]['Resolution']})")
    print(f"  Max: {np.max(brier_scores):.6f} ({df_display.iloc[np.argmax(brier_scores)]['Model']} @ {df_display.iloc[np.argmax(brier_scores)]['Resolution']})")
    
    mean_preds = df_display['Mean Pred Prob'].values
    mean_trues = df_display['Mean True Label'].values
    calibration_gaps = np.abs(mean_preds - mean_trues)
    print(f"\nAvg calibration gap (|pred - true|): {np.mean(calibration_gaps):.4f}")
    print(f"  Min gap: {np.min(calibration_gaps):.4f}")
    print(f"  Max gap: {np.max(calibration_gaps):.4f}")
    
    print("\n" + "=" * 120)



CALIBRATION METRICS SUMMARY TABLE
          Model Resolution  Brier Score  N Samples  Mean Pred Prob  Mean True Label
         1D-CNN     62.5Hz     0.010784       2185        0.142910         0.133181
Hybrid CNN+LSTM     62.5Hz     0.015501       2185        0.144664         0.133181
         1D-CNN      125Hz     0.013038       2185        0.145575         0.133181
Hybrid CNN+LSTM      125Hz     0.012609       2185        0.144818         0.133181
         1D-CNN      250Hz     0.013949       2185        0.145526         0.133181
Hybrid CNN+LSTM      250Hz     0.016043       2185        0.144139         0.133181
         1D-CNN      500Hz     0.028855       2185        0.164796         0.133181
Hybrid CNN+LSTM      500Hz     0.015197       2185        0.152052         0.133181

INTERPRETATION GUIDE

1. BRIER SCORE (lower is better):
   • Mean squared error between predicted probability and true label
   • Range: 0 (perfect) to 1 (worst)
   • Typical clinical models: 0.1–0.3
   • Int